## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

__QUESTIONS:
Q10-13 below__




In [147]:
import sqlite3
import pandas as pd
import numpy as np



conn = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = conn.cursor()

cur.execute("select name from sqlite_master where type = 'table';")

rows = cur.fetchall()
print(rows)


[('Bookings',), ('Facilities',), ('Members',)]


## Q10 

**Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!**

In [81]:

cols = ['facility_name', 'total_revenue']
query_result = cur.execute("select F.name, F.total_revenue FROM (SELECT Y.name, COUNT(X.facility_name) as num_months, SUM(X.revenue), Y.monthlymaintenance, (COUNT(X.facility_name) * Y.monthlymaintenance) as total_cost, (SUM(X.revenue) + (COUNT(X.facility_name) * Y.monthlymaintenance)) as total_revenue  FROM (select strftime('%m-%Y', starttime) as 'month-year', facility_name, SUM(revenue) as revenue FROM (select b.bookid, starttime, f.name as facility_name, b.memid, membercost, guestcost, slots, CASE WHEN memid = 0 THEN (slots * guestcost) ELSE (slots * membercost) END AS revenue from bookings AS b join facilities AS f ON b.facid = f.facid) GROUP BY strftime('%m-%Y', starttime), facility_name) AS X JOIN facilities AS Y ON X.facility_name = Y.name GROUP BY X.facility_name) AS F WHERE F.total_revenue < 1000 ORDER BY F.total_revenue DESC;").fetchall()


rev_less_1000 = pd.DataFrame(query_result, columns = cols)

print(rev_less_1000)

   facility_name  total_revenue
0     Pool Table            315
1  Snooker Table            285
2   Table Tennis            210


## Q11 

**Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [92]:
cols = ['member_surname', 'member_firstname', 'recommended_by']
query_result = cur.execute("select m.surname, m.firstname, (X.surname || ', ' || X.firstname) from members AS m LEFT JOIN members as X ON m.recommendedby = X.memid ORDER BY m.surname, m.firstname;").fetchall()


members_w_recs = pd.DataFrame(query_result, columns = cols)
print(members_w_recs)

       member_surname member_firstname      recommended_by
0               Bader         Florence    Stibbons, Ponder
1               Baker             Anne    Stibbons, Ponder
2               Baker          Timothy     Farrell, Jemima
3              Boothe              Tim         Rownam, Tim
4             Butters           Gerald       Smith, Darren
5              Coplin             Joan      Baker, Timothy
6             Crumpet            Erica        Smith, Tracy
7                Dare            Nancy    Joplette, Janice
8             Farrell            David                None
9             Farrell           Jemima                None
10              GUEST            GUEST                None
11            Genting          Matthew     Butters, Gerald
12               Hunt             John  Purview, Millicent
13              Jones            David    Joplette, Janice
14              Jones          Douglas        Jones, David
15           Joplette           Janice       Smith, Darr

## Q12

**Q12: Find the facilities with their usage by member, but not guests**

In [109]:
cols = ['facility_name', 'member_bookings_count']
query_result = cur.execute("select f.name, SUM(b.slots) as member_bookings from bookings AS b JOIN facilities AS f ON b.facid = f.facid WHERE b.memid != 0 GROUP BY b.facid ORDER BY member_bookings DESC;").fetchall()


bookings = pd.DataFrame(query_result, columns = cols)
print(bookings)



     facility_name  member_bookings_count
0  Badminton Court                   1086
1   Tennis Court 1                    957
2   Massage Room 1                    884
3   Tennis Court 2                    882
4    Snooker Table                    860
5       Pool Table                    856
6     Table Tennis                    794
7     Squash Court                    418
8   Massage Room 2                     54


## Q13

**Q13: Find the facilities usage by month, but not guests**

In [146]:
query_result = cur.execute("PRAGMA table_info('Bookings');").fetchall()
#print(query_result)

bookings_col_raw = pd.DataFrame(query_result)
bookings_col_names = bookings_col_raw[1]

cols = ['facility_name', 'month-year', 'member_booking_count']
query_result = cur.execute("select f.name, strftime('%m-%Y', starttime) as 'month_year', SUM(slots) as usage from bookings AS b JOIN facilities AS F on b.facid = f.facid WHERE memid != 0 GROUP BY strftime('%m-%Y', starttime), f.name ORDER BY usage DESC;").fetchall()

bookings = pd.DataFrame(query_result, columns = cols)
print(bookings)

      facility_name month-year  member_booking_count
0   Badminton Court    09-2012                   507
1        Pool Table    09-2012                   443
2    Tennis Court 1    09-2012                   417
3   Badminton Court    08-2012                   414
4    Tennis Court 2    09-2012                   414
5     Snooker Table    09-2012                   404
6    Massage Room 1    09-2012                   402
7      Table Tennis    09-2012                   400
8    Tennis Court 2    08-2012                   345
9    Tennis Court 1    08-2012                   339
10   Massage Room 1    08-2012                   316
11    Snooker Table    08-2012                   316
12       Pool Table    08-2012                   303
13     Table Tennis    08-2012                   296
14   Tennis Court 1    07-2012                   201
15     Squash Court    08-2012                   184
16     Squash Court    09-2012                   184
17   Massage Room 1    07-2012                